# Subjectivity classification using a pretrained language model

In this notebook we implement the approched described in cite... Jeremy's paper In particular, we will classify sentences into "subjective" or "objective". 

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [28]:
import os

## Subjectivity Dataset

The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
from pathlib import Path
PATH = Path("/data/yinterian/rotten_imdb/")
list(PATH.iterdir())

[PosixPath('/data/yinterian/rotten_imdb/subjdata.README.1.0'),
 PosixPath('/data/yinterian/rotten_imdb/plot.tok.gt9.5000'),
 PosixPath('/data/yinterian/rotten_imdb/quote.tok.gt9.5000')]

From the readme file:
- quote.tok.gt9.5000 contains 5000 subjective sentences (or snippets)
- plot.tok.gt9.5000 contains 5000 objective sentences

In [3]:
! head /data/yinterian/rotten_imdb/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 
spurning her mother's insistence that she get on with her life , mary is thrown out of the house , rejected by joe , and expelled from school as she grows larger with child . 
amitabh can't believe the board of directors and his mind is filled with revenge and what better revenge than robbing the bank himself , ironic as it may sound . 
she , among others excentricities , talks to a small rock , gertrude , like if she was alive . 
this gives the girls a fair chance of pulling the wool over their eyes using their sexiness to poach any last vestige of common sense the dons might have had . 
styled after vh1's " behin

## String cleaning functions

In [5]:
import numpy as np
from collections import defaultdict
import re

In [10]:
import re
# this is from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [11]:
def read_file(path):
    """ Read file returns a shuttled list.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = np.array(f.readlines())
    return content

In [18]:
from collections import defaultdict 
def get_vocab(list_of_content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for content in list_of_content:
        for line in content:
            line = clean_str(line.strip())
            words = set(line.split())
            for word in words:
                vocab[word] += 1
    return vocab       

## Split train and test

In [13]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([clean_str(line.strip()) for line in sub_content])
obj_content = np.array([clean_str(line.strip()) for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path \\?',
        "the director 's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship \\( most notably wretched sound design \\)",
        "welles groupie scholar peter bogdanovich took a long time to do it , but he 's finally provided his own broadside at publishing giant william randolph hearst",
        'based on the 1997 john king novel of the same name with a rather odd synopsis a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack'],
       dtype='<U679'), array([1., 0., 0., 1., 1.]))

In [19]:
# getting vocab from training sets
vocab = get_vocab([X_train])

## Laguage Model

In [20]:
class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)
        self.init_weights()
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        init_range = 0.1
        self.encoder.weight.data.uniform_(-init_range, init_range)
        self.decoder.bias.data.fill_(0.0)
        self.decoder.weight.data.uniform_(-init_range, init_range)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.nlayers, bsz, self.nhid).zero_())

In [22]:
nemb = 300
nhid = 300
nlayers = 2
ntokens = 33278
lang_model = RNNModel(ntokens, nemb, nhid, nlayers).cuda()

In [24]:
## load pre-trained model
model_path = "/data/yinterian/wikitext-2/mode117.pth"
def load_model(m, p):
    m.load_state_dict(torch.load(p))
load_model(lang_model, model_path)

In [23]:
lang_model

RNNModel (
  (drop): Dropout (p = 0.5)
  (encoder): Embedding(33278, 300)
  (rnn): GRU(300, 300, num_layers=2, dropout=0.5)
  (decoder): Linear (300 -> 33278)
)

## Encoding data with original corpus

In [32]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path, train_file, valid_file, test_file=None, word2idx=None):
        self.dictionary = Dictionary()
        self.word2idx = word2idx
        if word2idx is None:
            tr_tokens = self.add_words(os.path.join(path, train_file))
            val_tokens = self.add_words(os.path.join(path, valid_file))
            test_tokens = self.add_words(os.path.join(path, test_file))
            self.word2idx = self.dictionary.word2idx
        self.train = self.tokenize(os.path.join(path, train_file), tr_tokens)
        self.valid = self.tokenize(os.path.join(path, valid_file), val_tokens)
        self.test = self.tokenize(os.path.join(path, test_file), test_tokens)
        
    def add_words(self, path):
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        return tokens

    def tokenize(self, path, tokens):
        """Tokenizes a text file."""
        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.word2idx[word]
                    token += 1
        return ids

In [33]:
original_corpus = Corpus("/data/yinterian/wikitext-2","wiki.train.tokens", "wiki.valid.tokens", "wiki.test.tokens")

In [34]:
original_corpus.word2idx

{'<eos>': 0,
 '=': 1,
 'Valkyria': 2,
 'Chronicles': 3,
 'III': 4,
 'Senjō': 5,
 'no': 6,
 '3': 7,
 ':': 8,
 '<unk>': 9,
 '(': 10,
 'Japanese': 11,
 '戦場のヴァルキュリア3': 12,
 ',': 13,
 'lit': 14,
 '.': 15,
 'of': 16,
 'the': 17,
 'Battlefield': 18,
 ')': 19,
 'commonly': 20,
 'referred': 21,
 'to': 22,
 'as': 23,
 'outside': 24,
 'Japan': 25,
 'is': 26,
 'a': 27,
 'tactical': 28,
 'role': 29,
 '@-@': 30,
 'playing': 31,
 'video': 32,
 'game': 33,
 'developed': 34,
 'by': 35,
 'Sega': 36,
 'and': 37,
 'Media.Vision': 38,
 'for': 39,
 'PlayStation': 40,
 'Portable': 41,
 'Released': 42,
 'in': 43,
 'January': 44,
 '2011': 45,
 'it': 46,
 'third': 47,
 'series': 48,
 'same': 49,
 'fusion': 50,
 'real': 51,
 'time': 52,
 'gameplay': 53,
 'its': 54,
 'predecessors': 55,
 'story': 56,
 'runs': 57,
 'parallel': 58,
 'first': 59,
 'follows': 60,
 '"': 61,
 'Nameless': 62,
 'penal': 63,
 'military': 64,
 'unit': 65,
 'serving': 66,
 'nation': 67,
 'Gallia': 68,
 'during': 69,
 'Second': 70,
 'Europan

## Encoding training and validation sets

In [37]:
word2idx = original_corpus.word2idx

In [40]:
word2idx["half"]

994

In [36]:
X_train[0] + ' <eos>'

'will god let her fall or give her a new path \\? <eos>'

In [41]:
# returns the index of the word or the index of "UNK" otherwise
word2idx.get("will", 0)

300

In [43]:
np.array([word2idx.get(w, 0) for w in X_train[0].split()])

array([ 300, 5010, 7291,  361, 6356,  310, 2193,  361,   27,  578, 9428,
          0])

In [47]:
def encode_sentence(s, word2idx=word2idx, N=35):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([word2idx.get(w, 0) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [48]:
encode_sentence(X_train[0])

array([ 300, 5010, 7291,  361, 6356,  310, 2193,  361,   27,  578, 9428,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)

In [49]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 35)

In [50]:
x_test = np.vstack([encode_sentence(x) for x in X_test])
x_test.shape

(2000, 35)

## Transfer learning from language model

## Training

Note that I am not bodering with mini-batches since our dataset is small.

In [337]:
#def train(model, x_train, y_train):
model = SentenceCNN(V, D, glove_weights=pretrained_weight).cuda()

In [338]:
test_metrics(model)

test loss 0.698 and accuracy 0.490


In [339]:
# this filters parameters with p.requires_grad=True
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [340]:
def train_epocs(model, epochs=10, lr=0.01):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    model.train()
    for i in range(epochs):
        x = Variable(torch.LongTensor(x_train)).cuda()
        y = Variable(torch.Tensor(y_train)).cuda().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.data[0])
    test_metrics(model)

In [341]:
def test_metrics(m):
    model.eval()
    x = Variable(torch.LongTensor(x_test)).cuda()
    y = Variable(torch.Tensor(y_test)).cuda().unsqueeze(1)
    y_hat = m(x)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/pred.shape[0]
    print("test loss %.3f and accuracy %.3f" % (loss.data[0], accuracy.data[0]))

In [342]:
train_epocs(model, epochs=10, lr=0.01)

0.7035446763038635
1.1868393421173096
0.9086019396781921
0.5865945816040039
0.3559541702270508
0.3733510375022888
0.4246615469455719
0.4422186017036438
0.4157627522945404
0.3762800097465515
test loss 0.360 and accuracy 0.860


In [343]:
train_epocs(model, epochs=10, lr=0.01)

0.34370580315589905
0.5294861793518066
0.32328617572784424
0.40581214427948
0.2806266248226166
0.2568565309047699
0.30520763993263245
0.29174017906188965
0.24017520248889923
0.22176489233970642
test loss 0.285 and accuracy 0.892


In [344]:
train_epocs(model, epochs=10, lr=0.001)

0.23691332340240479
0.2138260155916214
0.20660893619060516
0.20420822501182556
0.20270921289920807
0.20497538149356842
0.19893155992031097
0.1915552318096161
0.19009748101234436
0.18447645008563995
test loss 0.246 and accuracy 0.898


In [345]:
# how to figure out the parameters
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


In [346]:
# unfreezing the embeddings
model.embedding.weight.requires_grad = True

In [347]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
print([p.size() for p in parameters])

[torch.Size([400007, 300]), torch.Size([100, 300, 3]), torch.Size([100]), torch.Size([100, 300, 4]), torch.Size([100]), torch.Size([100, 300, 5]), torch.Size([100]), torch.Size([1, 300]), torch.Size([1])]


In [348]:
train_epocs(model, epochs=10, lr=0.001)

0.1830516755580902
0.17343704402446747
0.16393160820007324
0.15581797063350677
0.14433661103248596
0.1368999481201172
0.1289738416671753
0.12384654581546783
0.11491061747074127
0.1075991541147232
test loss 0.228 and accuracy 0.913


In [349]:
train_epocs(model, epochs=10, lr=0.0001)

0.10160469263792038
0.10249238461256027
0.10043996572494507
0.10032773017883301
0.09892096370458603
0.09907413274049759
0.09923809766769409
0.09644830971956253
0.09582842141389847
0.0971798449754715
test loss 0.227 and accuracy 0.912


## TODOs

* Show how to save model
* Show how to predict on new data
* Test a version with a smaller word embedding matrix
* Try Another tokenizer

## Lab 

* You may not need to keep all word embeddings.
* Extend this code by finetunning the embedding layer.
* Use fasttext instead of globe model. (https://fasttext.cc/docs/en/english-vectors.html)

   `! pip install git+https://github.com/facebookresearch/fastText.git`
* Extend this code to do cross-validation. Look at https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py for an example on how to do it.

## References

* Language model from here https://github.com/pytorch/examples/blob/master/word_language_model/model.py